#### Imports

In [ ]:
# Visualization
import matplotlib.pyplot as plt

# Builtin
from pathlib import Path
import sys
from math import nan
from itertools import islice
import subprocess
import shutil
import os
import pdb


# Add repo path to system path for additional imports. 
# Note the relative depth from this notebook to system repo
# in case you want to move this file

nb_path = str(Path(os.path.abspath('')).parent.parent)
sys.path.insert(0, nb_path)
print(f'Added {nb_path} to system path')

# This computer git repos
from project.project_manager_module import ProjectManager

#### Build paths

In [ ]:
#root_path = r"/opt2/Git_Repos/SystemTools/scripts/"
root_path = r"C:\Users\Simo\Laskenta\Git_Repos\SystemTools\scripts/"
project = "FCN22"  
experiment = "Figure1B"  
experiment_path = Path.joinpath(Path(root_path), Path(project), experiment)

#### Input context

In [ ]:
input_folder = "../in" # Relative to experiment path
matlab_workspace_file = "matlab_workspace_SingleSpike.mat"
conn_skeleton_file_in = "Replica_skeleton_connections_20210211_1453238_L4_SS.gz"
conn_file_out = "connections_SingleSpike_ci.gz"
input_filename = "input_quadratic_three_units_2s_SingleSpike.mat"  
startpoint_csv_folder = "startpoint_csvs/LIF"

#### Data context for single files and arrays 
These startpoint and parameter strings are used only in this module.
The output_folder will be created under experiment

In [ ]:
startpoint = "Comrad"  # HiFi, Comrad, Bacon
parameter = "sinewave"
output_folder = f"{startpoint}_{parameter}"

#### Call project manager
Code complexity is hidden behind the ProjectManager

In [ ]:
PM = ProjectManager(
    path=experiment_path,
    input_folder=input_folder,
    output_folder=output_folder,
    matlab_workspace_file=matlab_workspace_file,
    conn_skeleton_file_in=conn_skeleton_file_in,
    conn_file_out=conn_file_out,
    input_filename=input_filename,
    NG_name="NG3_L4_SS_L4",
    t_idx_start=2000,
    t_idx_end=-2000,
    project=project,
    experiment=experiment,
    startpoint_csv_folder=startpoint_csv_folder,
)

#### Copy Anat and Phys csv files from start point folder to path and update paths into the csv:s

In [ ]:
anat_file_fullpath_out , phys_file_fullpath_out = PM.prepare_csvs_for_simulation()

#### Create necessary current injection files

In [ ]:
PM.ct.create_current_injection(randomize=False)

#### Run CxSystem2 for single file

In [ ]:
command_and_args = [
    "cxsystem2", 
    "-a", 
    str(anat_file_fullpath_out), 
    "-p", 
    str(phys_file_fullpath_out)
]

subprocess.run(command_and_args, text=True, encoding="utf-8")

### Analyze and visualize the results for single files

#### Show input and data. 
If  file_to_display = None, the function selects the most recent data file in output_folder.


In [ ]:
# file_to_display = r"[your/full/path/to/datafile].gz"  
file_to_display = None # Most recent

PM.viz.show_readout_on_input(
    results_filename=file_to_display, 
    normalize=False, 
    unit_idx_list=[0], 
    savefigname='Fig1B.eps'
)